# Version 2 (sparse matrices...)
### I created the last notebook naively and completely missed that the discusses method operates on sparse matrices, so I'm starting again from scratch

# Generating a sparse matrix representation of a graph...

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# First we generate some arbitrary graph
def generate_graph(num_nodes, average_out_degree, total_edge_classes):
    num_edges = average_out_degree*num_nodes
    indices = torch.randint(0, num_nodes, (2, num_edges))
    print(indices.size())
    edges = torch.ones(num_edges)
    print(edges.size())
    A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes))
    return A
# A = generate_graph(15,3,5).to_dense()

# Now maybe it's useful to ensure it's connected...
def generate_connected_graph(num_nodes, average_out_degree, total_edge_classes):
    edges_set = set()
    num_edges = average_out_degree * num_nodes

    def sample_random_edge_class():
        return torch.randint(0, total_edge_classes, (1,)).item()
    
    def generate_random_edge():
        u, v = torch.randint(0, num_nodes, (2,)).tolist()
        #no self loops
        while u == v:
            u, v = torch.randint(0, num_nodes, (2,)).tolist()
        return (u, v)
    
    def generate_random_node():
        return torch.randint(0, num_nodes, (1,)).item()
    
    def sample_random_node(A):
        connected_nodes = A._indices()[1].tolist() #select from incoming to...
        if not connected_nodes:
            return generate_random_node()
        return connected_nodes[torch.randint(0, len(connected_nodes), (1,)).item()]#torch.nonzero(A)
    
    initial_edge = generate_random_edge()
    edges_set.add(initial_edge)
    indices = torch.tensor([[initial_edge[0]], [initial_edge[1]]], dtype=torch.long, device=device)
    edges = torch.ones(1, device=device)
    A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes), device=device)
    
    for _ in range(num_edges - 1):
        u = sample_random_node(A)
        v = generate_random_node()
        while u == v:
            v = generate_random_node()
        new_edge = (u, v)#torch.cat([u, v], dim=0).unsqueeze(1)
        if new_edge in edges_set:
            continue
        edges_set.add(new_edge)
        edge_tensor = torch.tensor([[u], [v]], dtype=torch.long, device=device)
        indices = torch.cat([indices, edge_tensor], dim=1)
        edges = torch.ones(indices.size(1), device=device)
        A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes), device=device)
        # indices = torch.stack((indices,torch.cat([u,v]).unsqueeze(1)),dim=0)
    # print(edges_set)
    
    print(A.to_dense())
    
A = generate_connected_graph(15,15,5)

# Now per edge type? or randomly assign edges...? maybe multinomial_sample(1/k)^k?

tensor([[0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.],
        [1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1.],
        [1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.],
        [1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.],
        [1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1.],
        [1., 1., 1.,